In [1]:
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool
import numpy as np
import pandas as pd
import make_data
import datetime
import requests
import glob
import sys
import os
import json
import time
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier, NearestNeighbors
from sklearn.cross_validation import cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import RidgeClassifier, LinearRegression, LogisticRegression

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# names = [91, 111, 106, 104, 101, 327, 331, 332, 335, 94, 346, 348, 352, 401, 798, 92, 245, 325, 97, 1198, 1124, 1126, 1205, 1206, 1183, 1204, 1186, 1187, 1188, 1190, 1191, 90, 147, 159, 161, 1080, 157, 1081, 1100, 1101, 1102, 1103, 150, 173, 174, 175, 443, 1099, 154, 176, 177, 178, 179, 180, 156, 158, 162, 163, 1104, 1237, 164, 165, 166, 167, 916, 919, 168, 169, 170, 171, 172, 917, 918, 1105, 89, 141, 142, 143, 139, 136, 137, 138, 133, 135, 134, 131, 126, 130, 129, 127, 128, 123, 125, 124, 116, 122, 119, 121, 120, 117, 118, 936, 105, 114, 115, 113, 112, 110, 108, 107, 937, 103, 93, 98, 102, 99, 100, 96, 95, 132, 144, 145, 146, 148, 149, 151, 152, 153, 457, 458, 637, 638, 459, 460, 461, 639, 640, 462, 464, 641, 466, 595, 596, 597, 636, 181, 182, 193, 200, 446, 197, 218, 219, 220, 221, 222, 223, 224, 225, 198, 201, 235, 202, 240, 243, 448, 449, 451, 452, 203, 204, 445, 970, 226, 228, 229, 230, 231, 227, 199, 249, 250, 263, 274, 276, 277, 1388, 1422, 292, 1389, 303, 306, 307, 308, 1384, 1385, 1386, 1387, 310, 312, 314, 316, 1424, 1145, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430, 431, 1189, 432, 433, 435, 436, 437, 439, 442, 453, 454, 1079, 705, 194, 196, 210, 1499, 1500, 1501, 706, 707, 708, 709, 711, 725, 712, 713, 714, 715, 716, 717, 767, 768, 1143, 1214, 1345, 720, 721, 722, 1213, 723, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 560, 563, 564, 566, 570, 589, 1074, 576, 585, 578, 586, 579, 587, 583, 590, 591, 592, 1082, 598, 508, 509, 535, 510, 538, 1463, 539, 540, 547, 553, 554, 557, 573, 581, 1390, 892, 1219, 1377, 1234, 599, 605, 606, 607, 608, 609, 611, 610, 612, 601, 600, 603, 1464, 1465, 604, 1220, 1397, 1221, 1223, 613, 629, 1240, 1241, 1242, 1243, 1244, 1250, 1289, 614, 615, 616, 617, 618, 620, 621, 1327, 630, 1245, 1323, 1328, 631, 1320, 1321, 1322, 634, 1247, 1248, 1249, 1083, 619, 1292, 1294, 1084, 1326, 1286, 1288, 1290, 622, 623, 624, 625, 626, 627, 628, 635, 1291, 1295, 1299, 1316, 1317, 632, 1246, 1252, 1253, 1254, 1255, 633, 1296, 1297, 1298, 1318, 1319, 1324, 1325, 1287, 1293, 1236, 1376, 1455, 1456, 1457, 1458, 1461, 1462, 642, 643, 648, 649, 1182, 650, 959, 651, 652, 1157, 653, 654, 759, 644, 655, 1158, 656, 657, 658, 645, 659, 660, 1227, 647, 664, 1235, 1348, 1378, 1353, 1356, 1355, 1360, 1362, 1365, 1366, 1367, 1368, 1382, 1383, 1369, 1370, 1371, 1372, 661, 646, 662, 663, 799, 1132, 1133, 668, 669, 674, 1229, 672, 673, 676, 677, 695, 697, 698, 699, 700, 670, 678, 679, 680, 683, 701, 702, 703, 704, 681, 1215, 1216, 1217, 1315, 728, 729, 730, 732, 733, 769, 770, 771, 772, 773, 774, 775, 776, 1222, 734, 738, 739, 762, 743, 745, 747, 748, 764, 744, 749, 754, 755, 757, 750, 752, 753]
names = [91, 111, 106, 104, 101, 327, 331, 332, 335, 94, 346, 348, 352, 401, 798, 92, 245, 325, 97, 1198, 1124, 1126, 1205, 1206, 1183, 1204, 1186, 1187, 1188, 1190, 1191, 90, 147, 159, 161, 1080, 157, 1081, 1100, 1101, 1102, 1103, 150, 173, 174, 175, 443, 1099, 154, 176, 177, 178, 179, 180, 156, 158, 162, 163, 1104, 1237, 164, 165, 166, 167, 916, 919, 168, 169, 170, 171, 172, 917, 918, 1105, 89, 141, 142, 143, 139, 136, 137, 138, 133, 135, 134, 131, 126, 130, 129, 127, 128, 123, 125, 124, 116, 122, 119, 121, 120, 117, 118, 936, 105, 114, 115, 113, 112, 110, 108, 107, 937, 103, 93, 98, 102, 99, 100, 96, 95, 132, 144, 145, 146, 148, 149, 151, 152, 153, 457, 458, 637, 638, 459, 460, 461, 639, 640, 462, 464, 641, 466, 595, 596, 597, 636, 181, 182, 193, 200, 446, 197, 218, 219, 220, 221, 222, 223, 224, 225, 198, 201, 235, 202, 240, 243, 448, 449, 451, 452, 203, 204, 445, 970, 226, 228, 229, 230, 231, 227, 199, 249, 250, 263, 274, 276, 277, 1388, 1422, 292, 1389, 303, 306, 307, 308, 1384, 1385, 1386, 1387, 310, 312, 314, 316, 1424, 1145, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430, 431, 1189, 432, 433, 435, 436, 437, 439, 442, 453, 454, 1079, 705, 194, 196, 210, 1499, 1500, 1501, 706, 707, 708, 709, 711, 725, 712, 713, 714, 715, 716, 717, 767, 768, 1143, 1214, 1345, 720, 721, 722, 1213, 723, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 560, 563, 564, 566, 570, 589, 1074, 576, 585, 578, 586, 579, 587, 583, 590, 591, 592, 1082, 598, 508, 509, 535, 510, 538, 1463, 539, 540, 547, 553, 554, 557, 573, 581, 1390, 892, 1219, 1377, 1234, 601, 600, 603, 1464, 1465, 604, 1220, 1397, 1221, 1223, 613, 629, 1240, 1241, 1242, 1243, 1244, 1250, 1289, 614, 615, 616, 617, 618, 620, 621, 1327, 630, 1245, 1323, 1328, 631, 1320, 1321, 1322, 634, 1247, 1248, 1249, 1083, 619, 1292, 1294, 1084, 1326, 1286, 1288, 1290, 622, 623, 624, 625, 626, 627, 628, 635, 1291, 1295, 1299, 1316, 1317, 632, 1246, 1252, 1253, 1254, 1255, 633, 1296, 1297, 1298, 1318, 1319, 1324, 1325, 1287, 1293, 1236, 1376, 1461, 1462, 642, 643, 648, 649, 1182, 650, 959, 651, 652, 1157, 653, 654, 759, 644, 655, 1158, 656, 657, 658, 645, 659, 660, 1227, 647, 664, 1235, 1348, 1378, 1353, 1356, 1355, 1360, 1362, 1365, 1366, 1367, 1368, 1382, 1383, 1369, 1370, 1371, 1372, 661, 646, 662, 663, 799, 1132, 1133, 668, 669, 674, 1229, 672, 673, 676, 677, 695, 697, 698, 699, 700, 670, 678, 679, 680, 683, 701, 702, 703, 704, 681, 1215, 1216, 1217, 1315, 728, 729, 730, 732, 733, 769, 770, 771, 772, 773, 774, 775, 776, 1222, 734, 738, 739, 762, 743, 745, 747, 748, 764, 744, 749, 754, 755, 757, 750, 752, 753]

In [4]:
tree = np.load('data/tree.npy')

In [5]:
def dfs(tree, parents, childrens):
    mas = []
    if tree['children'] != None:
        for i in tree['children']:
            parents[i['a_attr']['title'].split('.')[0]] = tree['a_attr']['title'].split('.')[0]
            mas.extend([i['a_attr']['title'].split('.')[0]])
            dfs(i, parents, childrens)
    childrens[tree['a_attr']['title'].split('.')[0]] = mas
    
parents = {'0':'0'}
childrens = {}
for i in tree:
    parents[i['a_attr']['title'].split('.')[0]] = '0'
    dfs(i, parents, childrens)

In [6]:
def getData(tree, names, allUrls, parent_numb, level, depth):
    dataUrls = []
    dataText = []
    dataCategories = []
    
    number = int(tree['a_attr']['title'].split('.')[0])
    parent = number
    if level < depth:
        parent = parent_numb
        
    
    flag = True
    if len(tree['children']) != 0:
        for i in tree['children']:
            a, b, c = getData(i, names, allUrls, parent, level, depth+1)
            if level - 2 == depth and len(c) != 0:
                dataUrls.append(a)
                dataText.append(b)
                dataCategories.append(c)
            else:
                dataUrls.extend(a)
                dataText.extend(b)
                dataCategories.extend(c)
                
    elif level - 1 == depth:
        flag = False
#     else:
#         if number in names and (len(categUrls) - len(dataUrls)) / len(categUrls) > 0.8:
#             dataCategories = (np.ones(len(dataCategories)) * parent_numb).tolist()
    
    if number in names and level - 1 <=  depth and flag and number in files:
        f = open('data/filterData/categories/{}.txt'.format(number))
        lines = f.readlines()
        categUrls = [i.split(', ', 1)[0] for i in lines]
        categText = [i.split(', ', 1)[1] for i in lines]
        f.close()
        
        for i in range(len(categUrls)):
            if categUrls[i] not in allUrls:
                dataUrls.extend([categUrls[i]])
                dataText.append([categText[i].replace('\n', '')])
                dataCategories.extend([parent])

                allUrls.add(categUrls[i])
    
    return dataUrls, dataText, dataCategories

In [7]:
files = glob.glob('data/filterData/categories/*')
files = [int(i[27:].split('.')[0]) for i in files]

In [8]:
allLevels = {}

In [9]:
for level in range(1,6):
    dataUrls = []
    dataText = []
    dataCategories = []
    parent_numb = 1
    for i in tree:
        allUrls = set()
        a, b, c = getData(i, names, allUrls, parent_numb, level, 0)
        if level == 1 and len(a) != 0:
            dataUrls.append(a)
            dataText.append(b)
            dataCategories.append(c)
        else:
            dataUrls.extend(a)
            dataText.extend(b)
            dataCategories.extend(c)

    for i in range(len(dataUrls)):
#         print(level, dataCategories[i][-1])
#         if len(set(dataCategories[i])) > 1:
#             if len(set(dataCategories[i])) > 2:
#                 catA = np.array(dataCategories[i]).copy()
#                 jj = np.max(np.where(catA != dataCategories[i][-1])[0]) + 1
#                 catA[:jj] = dataCategories[i][-1] * 1000
#                 catA = catA.tolist()
#                 allLevels[dataCategories[i][-1]] = (dataUrls[i], dataText[i], catA)

#                 newUrls = dataUrls[i][:jj]
#                 newText = dataText[i][:jj]
#                 newCateg = dataCategories[i][:jj]

#                 allLevels[dataCategories[i][-1]*1000] = (newUrls, newText, newCateg)
#             elif len(set(dataCategories[i])) == 2:
#                 allLevels[dataCategories[i][-1]] = (dataUrls[i], dataText[i], dataCategories[i])
    
            if len(set(dataCategories[i])) > 2:
                catA = np.array(dataCategories[i]).copy()
                jj = np.max(np.where(catA != dataCategories[i][-1])[0]) + 1
                newUrls = dataUrls[i][:jj]
                newText = dataText[i][:jj]
                newCateg = dataCategories[i][:jj]

                allLevels[dataCategories[i][-1]] = (newUrls, newText, newCateg)

In [10]:
testAns = np.load('data/filterData/TestAns.npy')
testUrl = [i[0] for i in testAns]

In [11]:
testText = []
testCategories = []
for i in testUrl:
    txt = '/ '
    cat = 0
    for j in allLevels:
        if i in allLevels[j][0]:
            idx = allLevels[j][0].index(i)
            txt = allLevels[j][1][idx]
            cat = allLevels[j][2][idx]
            
            del allLevels[j][0][idx]
            del allLevels[j][1][idx]
            del allLevels[j][2][idx]
    
    testText.extend([txt])
    testCategories.extend([cat])
testText = np.array(testText, dtype='O')

In [12]:
features = {}
_time = time.time()
for k in allLevels:
    texts = allLevels[k][1]
    
    texts = [i[0] for i in texts]
    
    tfidfTransformer = TfidfVectorizer()
    tfidf = tfidfTransformer.fit_transform(texts)
    
    features[k] = (tfidf, tfidfTransformer, allLevels[k][2])

print(time.time() - _time)

40.982948303222656


In [13]:
features.keys()

dict_keys([91, 1186, 90, 89, 457, 181, 249, 405, 431, 705, 474, 560, 598, 642, 668, 728, 101, 94, 92, 1124, 147, 150, 154, 156, 141, 133, 126, 123, 116, 105, 93, 146, 458, 460, 595, 197, 202, 970, 263, 303, 310, 406, 410, 415, 421, 1028, 194, 706, 712, 716, 720, 1074, 508, 601, 613, 643, 644, 645, 647, 661, 669, 670, 1215, 729, 738, 743, 744, 1080, 1100, 178, 163, 164, 167, 172, 119, 117, 98, 226, 422, 510, 629, 1288, 1293, 648, 1229, 678, 730, 749, 750, 590, 614, 630, 631, 634, 1294, 622, 632, 633])

# CLF

In [14]:
clfRC = {}
clfNC = {}
clfKNN1 = {}
clfKNN2 = {}
clfSVC = {}
clfLR = {}

_time1 = time.time()
for i in features:
    print(i)
    _time = time.time()
    X, tfidfTransformer, y = features[i]
    
    clfLR[i] = LogisticRegression()
    clfLR[i].fit(X, y)

#     clfRC[i] = RidgeClassifier()
#     clfRC[i].fit(X, y)
# #     print('Fit time: ', time.time() - _time)

# # #     _time = time.time()
#     clfNC[i] = NearestCentroid()
#     clfNC[i].fit(X, y)
# # # #     print('Fit time: ', time.time() - _time)

# # #     _time = time.time()
#     clfKNN1[i] = KNeighborsClassifier()
#     clfKNN1[i].fit(X, y)
# # # #     print('Fit time: ', time.time() - _time)

# # #     _time = time.time()
#     clfKNN2[i] = KNeighborsClassifier(algorithm='brute', metric='cosine')
#     clfKNN2[i].fit(X, y)
# # # #     print('Fit time: ', time.time() - _time)

# # #     _time = time.time()
#     clfSVC[i] = SVC(gamma=0.5)
#     clfSVC[i].fit(X, y)
    print('Fit time: ', time.time() - _time)
print('All time: ', time.time() - _time1)

91
Fit time:  0.46376538276672363
1186
Fit time:  0.008456707000732422
90
Fit time:  1.2542307376861572
89
Fit time:  2.35546875
457
Fit time:  1.0687363147735596
181
Fit time:  2.8338711261749268
249
Fit time:  0.3680737018585205
405
Fit time:  8.44026517868042
431
Fit time:  1.3908617496490479
705
Fit time:  1.9962456226348877
474
Fit time:  1.133875846862793
560
Fit time:  0.8616647720336914
598
Fit time:  2.587946653366089
642
Fit time:  0.5952162742614746
668
Fit time:  0.14145755767822266
728
Fit time:  0.5349612236022949
101
Fit time:  0.013296127319335938
94
Fit time:  0.004707813262939453
92
Fit time:  0.0029158592224121094
1124
Fit time:  0.005318641662597656
147
Fit time:  0.17838668823242188
150
Fit time:  0.19660377502441406
154
Fit time:  0.16351771354675293
156
Fit time:  0.6221621036529541
141
Fit time:  0.0015401840209960938
133
Fit time:  0.007988452911376953
126
Fit time:  0.015479087829589844
123
Fit time:  0.0048601627349853516
116
Fit time:  0.043462276458740234
1

In [35]:
predict = np.load('data/filterData/multilevel/predictLR.npy')
clf = clfLR
lr = 0.3
for i in clf:
    idx = np.where(predict == i)[0]
    if idx.shape[0] != 0:
        testX = testText[idx]
        testX = features[i][1].transform([j[0] for j in testX])

        predict[idx] = clf[i].predict(testX)
        
        idx2 = idx[np.where(clf[i].predict_proba(testX).max(axis=1)< lr)]
        predict[idx2] = i

sm = 0
print(accuracy_score(testCategories, predict))
for i in range(len(testCategories)):
    if testCategories[i] == predict[i] or \
        parents[str(int(testCategories[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategories[i]))]:
        sm += 1
print(sm / len(testCategories))

0.421058837401


KeyError: '0'

In [16]:
predict = np.load('data/filterData/multilevel/predictKNN1.npy')
clf = clfKNN1
for i in clf:
    idx = np.where(predict == i)[0]
    if idx.shape[0] != 0:
        testX = testText[idx]
        testX = features[i][1].transform([j[0] for j in testX])

        predict[idx] = clf[i].predict(testX)

sm = 0
print(accuracy_score(testCategories, predict))
for i in range(len(testCategories)):
    if testCategories[i] == predict[i] or \
        parents[str(int(testCategories[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategories[i]))]:
        sm += 1
print(sm / len(testCategories))

0.295130291239
0.33852140077821014


In [17]:
predict = np.load('data/filterData/multilevel/predictKNN2.npy')
clf = clfKNN2
for i in clf:
    idx = np.where(predict == i)[0]
    if idx.shape[0] != 0:
        testX = testText[idx]
        testX = features[i][1].transform([j[0] for j in testX])

        predict[idx] = clf[i].predict(testX)

sm = 0
print(accuracy_score(testCategories, predict))
for i in range(len(testCategories)):
    if testCategories[i] == predict[i] or \
        parents[str(int(testCategories[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategories[i]))]:
        sm += 1
print(sm / len(testCategories))

0.503478363401
0.6000471642494989


In [18]:
predict = np.load('data/filterData/multilevel/predictNC.npy')
clf = clfNC
for i in clf:
    idx = np.where(predict == i)[0]
    if idx.shape[0] != 0:
        testX = testText[idx]
        testX = features[i][1].transform([j[0] for j in testX])

        predict[idx] = clf[i].predict(testX)

sm = 0
print(accuracy_score(testCategories, predict))
for i in range(len(testCategories)):
    if testCategories[i] == predict[i] or \
        parents[str(int(testCategories[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategories[i]))]:
        sm += 1
print(sm / len(testCategories))

0.463152930079
0.5738710057776205


In [ ]:
predict = np.load('data/filterData/multilevel/predictRC.npy')
clf = clfRC
for i in clf:
    idx = np.where(predict == i)[0]
    if idx.shape[0] != 0:
        testX = testText[idx]
        testX = features[i][1].transform([j[0] for j in testX])

        predict[idx] = clf[i].predict(testX)

sm = 0
print(accuracy_score(testCategories, predict))
for i in range(len(testCategories)):
    if testCategories[i] == predict[i] or \
        parents[str(int(testCategories[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategories[i]))]:
        sm += 1
print(sm / len(testCategories))

0.555594859097
0.6513382855795307


In [ ]:
predict = np.load('data/filterData/multilevel/predictSVC1.npy')
clf = clfSVC
for i in clf:
    idx = np.where(predict == i)[0]
    if idx.shape[0] != 0:
        testX = testText[idx]
        testX = features[i][1].transform([j[0] for j in testX])

        predict[idx] = clf[i].predict(testX)

sm = 0
print(accuracy_score(testCategories, predict))
for i in range(len(testCategories)):
    if testCategories[i] == predict[i] or \
        parents[str(int(testCategories[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategories[i]))]:
        sm += 1
print(sm / len(testCategories))

# Matrix

In [15]:
predict = np.load('data/filterData/multilevel/predictRC.npy')
clf = clfRC
for i in clf:
    idx = np.where(predict == i)[0]
    if idx.shape[0] != 0:
        testX = testText[idx]
        testX = features[i][1].transform([i[0] for i in testX])

        predict[idx] = clf[i].predict(testX)

sm = 0
print(accuracy_score(testCategories, predict))
for i in range(len(testCategories)):
    if testCategories[i] == predict[i] or \
        parents[str(int(testCategories[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategories[i]))]:
        sm += 1
print(sm / len(testCategories))

0.561844122155
0.6595920292418347


In [16]:
categories = np.load('data/categor.npy').item()

In [18]:
# labels = list(set(trainCategory) | set(testCategory))
labels = names
matr = confusion_matrix(testCategories, predict, labels=labels)
for i in range(len(labels)):
#     print('Label:', labels[i], '\tAll:', matr[i].sum(), ' \tTrue:', matr[i, i], '\tMax:',\
#           labels[matr[i].argmax()], '\tMaxS:', matr[i].max(), '\tProc:', matr[i, i]/matr[i].sum())
#     if str(int(labels[i])) in categories:
    srt = matr[i].argsort()[::-1]
    print(categories[str(int(labels[i]))])
    print('All:', matr[i].sum(), ' \tPredict:', matr[i, i])
    for j in range(3):
        if srt[j] != i:
            print('Predict:', matr[i, srt[j]], '\t', categories[str(int(labels[srt[j]]))])
    print()

91. Компьютеры
All: 27  	Predict: 17
Predict: 3 	 111. Оргтехника (принтеры, сканеры, факсы)
Predict: 2 	 1124. Ремонт компьютеров

111. Оргтехника (принтеры, сканеры, факсы)
All: 47  	Predict: 43
Predict: 1 	 510. Автомобили. Запчасти
Predict: 1 	 581. Автомобильные аксессуары

106. Аксессуары
All: 1  	Predict: 0
Predict: 1 	 1046. Радиолюбители
Predict: 0 	 753. Отели
Predict: 0 	 249. Бытовая техника

104. Информационные носители и внешние устройства чтения
All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

101. Периферийные устройства
All: 2  	Predict: 0
Predict: 1 	 118. Counter-Strike
Predict: 1 	 1205. Советы по настройке и использованию ПК
Predict: 0 	 753. Отели

327. Клавиатуры, мыши, комплекты
All: 1  	Predict: 0
Predict: 1 	 553. Мультимедиа. Запчасти
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни

331. Компьютерная акустика(наушники, колонки) и микрофоны
All: 0  	Predict: 0
Predi

840. Азиатская кухня
All: 7  	Predict: 7
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

841. Восточная  кухня
All: 2  	Predict: 0
Predict: 2 	 422. Рестораны и кафе
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни

842. Латиноамериканская кухня
All: 1  	Predict: 0
Predict: 1 	 422. Рестораны и кафе
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни

843. Русская кухня
All: 2  	Predict: 0
Predict: 2 	 422. Рестораны и кафе
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни

884. Бары и пабы
All: 11  	Predict: 5
Predict: 6 	 422. Рестораны и кафе
Predict: 0 	 753. Отели

423. Бары
All: 5  	Predict: 1
Predict: 3 	 884. Бары и пабы
Predict: 1 	 422. Рестораны и кафе

424. Антикафе
All: 3  	Predict: 2
Predict: 1 	 422. Рестораны и кафе
Predict: 0 	 753. Отели

425. Клубы
All: 7  	Predict: 1
Predict: 4 	 422. Рестораны и кафе
Predict: 1 	 1289. Европейские авто

426. Отдых на природе
All: 20  	Predict: 5
Pre

All: 0  	Predict: 0
Predict: 0 	 753. Отели
Predict: 0 	 263. Мелкая техника для кухни
Predict: 0 	 292. Техника для красоты

633. Китайские авто
All: 4  	Predict: 0
Predict: 2 	 1290. Азиатские авто
Predict: 2 	 1289. Европейские авто
Predict: 0 	 753. Отели

1296. Chery
All: 4  	Predict: 0
Predict: 2 	 1289. Европейские авто
Predict: 1 	 1290. Азиатские авто
Predict: 1 	 510. Автомобили. Запчасти

1297. Lifan
All: 6  	Predict: 0
Predict: 4 	 1290. Азиатские авто
Predict: 1 	 510. Автомобили. Запчасти
Predict: 1 	 1289. Европейские авто

1298. Geely
All: 6  	Predict: 0
Predict: 6 	 1290. Азиатские авто
Predict: 0 	 753. Отели
Predict: 0 	 250. Климатическая техника

1318. FAW
All: 1  	Predict: 0
Predict: 1 	 1290. Азиатские авто
Predict: 0 	 753. Отели
Predict: 0 	 250. Климатическая техника

1319. GreatWall
All: 1  	Predict: 0
Predict: 1 	 1290. Азиатские авто
Predict: 0 	 753. Отели
Predict: 0 	 250. Климатическая техника

1324. Brilliance
All: 2  	Predict: 0
Predict: 2 	 1290. Азиа